# NSLS-II tardis diffractometer

The steps to setup *tardis* in **hklpy2** are described below. 

*tardis* is an environmental chamber with a 3-axis diffractometer, *operated
with* hkl_soleil solver's E6C geometry, with some of the real axes
[renamed](https://github.com/NSLS-II-CSX/profile_collection/blob/fc5d2bc5c542d83c2593b4f5066f52a5b04d748d/startup/csx1/startup/tardis.py#L31-L37).
Some E6C axes do not exist in *tardis*; they are fixed at zero.

The next schematic shows the *tardis* axes in the E6C geometry.  Energy units
are *eV*, wavelength units are *angstrom*.

![tardis schematic](../_static/nslsii-tardis.png)

## Create the `tardis` object.

In [1]:
import hklpy2

tardis = hklpy2.creator(
    name="tardis",
    geometry="E6C",
    solver="hkl_soleil",
    reals=dict(
        theta=None,  # None: use simulated motor
        mu=None,
        chi=None,
        phi=None,
        delta=None,
        gamma=None,
    ),
    labels=["tardis"],
)

Configure some basic operating parameters.

In [2]:
tardis.operator.solver.mode = "lifting_detector_mu"
tardis.operator.constraints["gamma"].limits = -5, 180
tardis._source.wavelength_units = "angstrom"
tardis._source.energy_units = "eV"
tardis.wavelength.put(13.317314715359827)

tardis.operator.constraints["theta"].limits = -181, 181
tardis.operator.constraints["delta"].limits = -5, 180
tardis.operator.constraints["gamma"].limits = -5, 180

# tardis does not have these axes, fix them at zero
tardis.operator.constraints["chi"].limits = 0, 0
tardis.operator.constraints["mu"].limits = 0, 0
tardis.operator.constraints["phi"].limits = 0, 0

## Add a sample

In [3]:
tardis.add_sample("KCF", 5.857, c=7.849)  # tetragonal

Sample(name='KCF', lattice=Lattice(a=5.857, c=7.849, system='tetragonal'))

## Orient

Add two observed reflections.

In [4]:
r1 = tardis.add_reflection(
    (0, 0, 1),
    dict(
        theta=48.42718305024724,
        mu=0.0,
        chi=0.0,
        phi=0.0,
        delta=115.65436271083637,
        gamma=3.0000034909999993,
    ),
    name="r1",
)
r2 = tardis.add_reflection(
    (1, 1, 0),
    dict(
        theta=138.42718305024724,
        mu=0.0,
        chi=0.0,
        phi=0.0,
        delta=115.65436271083637,
        gamma=3.0000034909999993,
    ),
    name="r2",
)

Calculate the $UB$ (orientation) matrix.

In [5]:
tardis.operator.calc_UB(r1, r2)
print(f"{tardis.operator.solver.UB=}")

tardis.operator.solver.UB=[[0.728792306003, 0.76737046489, -0.13102164758], [0.096473472661, 0.150408008639, 0.789324567316], [0.781267973381, -0.734401118598, 0.024752949397]]


## Save

In [6]:
tardis.export(
    "dev_tardis-kcf.yml",
    comment="NSLS-II tardis with oriented KCF sample",
)

Show that configuration.

In [7]:
%pycat dev_tardis-kcf.yml

#hklpy2 configuration file

_header:
  datetime: '2025-02-28 08:50:21.097209'
  hklpy2_version: 0.0.26.dev80+g8b020e4.d20250228
  python_class: Hklpy2Diffractometer
  source_type: X-ray
  energy_units: eV
  energy: 930.9999882751769
  wavelength_units: angstrom
  wavelength: 13.317314715359826
  file: dev_tardis-kcf.yml
  comment: NSLS-II tardis with oriented KCF sample
name: tardis
axes:
  pseudo_axes:
  - h
  - k
  - l
  real_axes:
  - theta
  - mu
  - chi
  - phi
  - delta
  - gamma
  axes_xref:
    h: h
    k: k
    l: l
    theta: mu
    mu: omega
    chi: chi
    phi: phi
    delta: gamma
    gamma: delta
  extra_axes: {}
sample_name: KCF
samples:
  sample:
    name: sample
    lattice:
      a: 1
      b: 1
      c: 1
      alpha: 90.0
      beta: 90.0
      gamma: 90.0
    reflections: {}
    reflections_order: []
    U: &id001
    - - 1
      - 0
      - 0
    - - 0
      - 1
      - 0
    - - 0
      - 0
      - 1
    UB: *id001
    digits: 4
  KCF:
    name: KCF
    lattice:

## Move

... to $Q= (0 \  0 \  1.1)$

In [8]:
tardis.move((0, 0, 1.1))

MoveStatus(done=True, pos=tardis, elapsed=0.0, success=True, settle_time=0.0)

Where is the *tardis* now?

In [9]:
tardis.wh()

h=0, k=0, l=1.1
wavelength=13.317314715359826
theta=101.5681, mu=0, chi=0, phi=0, delta=42.0222, gamma=176.6916
